必要モジュールのインストール

In [ ]:
!pip install shortuuid keras numpy==1.14.3 pandas==0.24.2 Pillow==5.2.0 requests==2.20.0 --target train/module

学習済みモデルのロード

In [ ]:
import sys
sys.path.append('train/module')
import utils

weight = utils.download_weiths()

定数の宣言

In [ ]:
import sys
sys.path.append('train/module')

#S3に作成されるフォルダの名前
PROJECT_NAME = 'sagemaker-with-keras-traing2deploy'
TAGS = [{ 'Key': 'example.ProjectName', 'Value': PROJECT_NAME }]
#tarinフォルダにある[train_xx.py]を指定する。xxの部分とVERSIONを一致させて使用する。
VERSION = 'v2'
#S3のバケット名　あらかじめ作成しておくこと
BUCKET_NAME = f'sage-maker-201908132134'
#S3からデータをダウンロードする際の設定（不使用）
DATA_ROOT = f's3://{BUCKET_NAME}/{PROJECT_NAME}'
#S3からデータをダウンロードする際の設定（不使用）
TRAINS_DIR = f'{DATA_ROOT}/data/trains'
#S3からデータをダウンロードする際の設定（不使用）
TESTS_DIR = f'{DATA_ROOT}/data/tests'
#S3に学習結果をアップロードする際の設定
OUTPUTS_DIR = f'{DATA_ROOT}/outputs'
#↓SageMakerコンソール ノートブック->Permissions and encryption->IAM ロール ARNのarnをコピペする
ROLE = 'arn:aws:iam::902023299513:role/service-role/AmazonSageMaker-ExecutionRole-20190812T024057'

パラメータの設定

In [ ]:
from sagemaker.tensorflow import TensorFlow
import logging

#train_xx.pyに渡す学習パラメータ
# batch-size:バッチサイズ
# epochs:エポック数
# lr:学習係数
# opt:オプティマイザー=最適化アルゴリズム
# depth:NN(ニューラルネットワーク)の隠れ層の数
# width:NNの各隠れ層のノードの数
# validation-split:教師データを学習用とテスト用に分ける比率。学習：テスト=入力値：(1-入力値)
# model-version:train_xx.pyのバージョン
# weight-file:学習結果の重みファイル
params = {
    'batch-size': 1,
    'epochs': 1,
    'lr':0.1,
    'opt':'sgd',
    'depth': 16,
    'width':8,
    'validation-split': 0.1,
    'model-version': VERSION,
    'weight-file':'pretrained_models/weights.28-3.73.hdf5'
}

# 学習状況を計測する指標
# CloudWotachに出力される
metric_definitions = [
    {'Name': 'train:acc', 'Regex': 'acc: (\S+)'},
    {'Name': 'train:mse', 'Regex': 'mean_squared_error: (\S+)'},
    {'Name': 'train:mae', 'Regex': 'mean_absolute_error: (\S+)'},
    {'Name': 'train:top-k', 'Regex': 'top_k_categorical_accuracy: (\S+)'},
    {'Name': 'valid:acc', 'Regex': 'val_acc: (\S+)'},
    {'Name': 'valid:mse', 'Regex': 'val_mean_squared_error: (\S+)'},
    {'Name': 'valid:mae', 'Regex': 'val_mean_absolute_error: (\S+)'},
    {'Name': 'valid:top-k', 'Regex': 'val_top_k_categorical_accuracy: (\S+)'},
]

#学習時に実行するTensorFlowのバージョンやインスタンスタイプの設定など
estimator = TensorFlow(
    role=ROLE,
    source_dir='train',
    entry_point=f'train_{VERSION}.py',
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge',
    framework_version='1.12.0',
    py_version='py3',
    script_mode=True,
    hyperparameters=params,
    dependencies=['train/module'],
    output_path=OUTPUTS_DIR,
    container_log_level=logging.INFO,
    metric_definitions=metric_definitions,
    tags=TAGS
)

#学習に渡すデータの設定（不使用）
#設定しておかないとエラーになるため一応設定している
inputs = {'train': TRAINS_DIR, 'test': TESTS_DIR}

学習

In [ ]:
import shortuuid

uuid = shortuuid.ShortUUID().random(length=8)
# estimator.fit(job_name=f'{PROJECT_NAME}-{VERSION}-s-{uuid}', inputs=inputs)
estimator.fit(job_name=f'{PROJECT_NAME}-{VERSION}-s-{uuid}', inputs=inputs)

学習結果のモデルのダウンロード

In [ ]:
import boto3
from urllib.parse import urlparse

#estimator = TensorFlow.attach(tuner.best_training_job())
#print(tuner.best_training_job())

url = urlparse(estimator.model_data)
s3_root_dir = '/'.join(url.path.split('/')[:-2])[1:]
model_s3path = s3_root_dir + '/output/model.tar.gz'
#output_s3path = s3_root_dir + '/output/output.tar.gz'
model_filename = 'predict/model_v1.h5'
s3 = boto3.resource('s3')
bucket = s3.Bucket(url.netloc)

print(model_s3path)
bucket.download_file(model_s3path, 'predict/model.tar.gz')
#bucket.download_file(output_s3path, 'predict/output.tar.gz')


#with open('models/labels.pickle', mode='rb') as f:
#    labels = pickle.load(f)


推論コンテナ用フォルダに解凍する

In [ ]:
!cd predict; tar zxvf model.tar.gz;
#!cd predict; tar tar zxvf output.tar.gz

ダウンロードしたモデルの入力と出力を確認

In [ ]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants

# TensorFlowによるモデルのロード
session = tf.keras.backend.get_session()
tf_model = tf.saved_model.loader.load(session, [tag_constants.SERVING], 'predict/1');

# input/outputのシグネチャ名確認
model_signature = tf_model.signature_def['serving_default']
input_signature = model_signature.inputs
output_signature = model_signature.outputs

for k in input_signature.keys():
    print(k)
for k in output_signature.keys():
    print(k)

推論コンテナ用に再パッケージ

In [ ]:
!cd predict; tar zcvf model.tar.gz 1 code

S3に推論用ソースをアップロード

In [ ]:
import urllib

predict_model_url = urllib.parse.urlparse(f'{estimator.output_path}/{estimator.latest_training_job.job_name}/predict/model.tar.gz')
bucket.upload_file('predict/model.tar.gz', predict_model_url.path[1:])

※estimatorのプロパティを使用しない場合のアップロードは↓

In [ ]:
import boto3
import urllib
from urllib.parse import urlparse

#estimator = TensorFlow.attach(tuner.best_training_job())
#print(tuner.best_training_job())
s3 = boto3.resource('s3')
bucket = s3.Bucket('sage-maker-201908132134')

predict_model_url = urllib.parse.urlparse(f's3://sage-maker-201908132134/sagemaker-with-keras-traing2deploy/outputs/sagemaker-with-keras-traing2deploy-v1-s-W9nCEBS5/predict/model.tar.gz')
bucket.upload_file('predict/model.tar.gz', predict_model_url.path[1:])

エンドポイントの作成

In [ ]:
from sagemaker.tensorflow.serving import Model

tensorflow_serving_model = Model(model_data=f'{predict_model_url.scheme}://{predict_model_url.hostname}{predict_model_url.path}',
                                 role=ROLE,
                                 framework_version='1.13')

In [ ]:
predictor = tensorflow_serving_model.deploy(initial_instance_count=1,
                                            instance_type='ml.t2.large',
                                            tags=TAGS)

In [ ]:
predict_model_url
estimator.latest_training_job.job_name

画像認識モデルを動作確認する

(送信側の作業)Base64エンコードの画像をJSONにシリアライズする

In [ ]:
import base64
def encodeToBase64(filename):
    with open(filename, "rb") as f:
        img_base64 = base64.b64encode(f.read())

    return img_base64

In [ ]:
import json

b64img = encodeToBase64('./train/data/ogurishun.jpg')
dcded_b64img = b64img.decode('utf-8')
query = {'image':dcded_b64img }
param = json.dumps(query)

画像送信

In [ ]:
client = boto3.client('sagemaker-runtime')
res = client.invoke_endpoint(
    EndpointName=predictor.endpoint,
    Body=param,
    ContentType='application/json',
    Accept='application/json'
)
body = res['Body'].read()

画損認識の結果を取得

In [ ]:
r = body.decode('utf-8')
x = json.loads(r)
x = x[0]['prob']
print(x)

In [ ]:
!pip install tqdm --target train/module

In [ ]:
!cd ./train; python ./evaluate_appa_real.py